In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import scipy as sp
import scipy.optimize
import seaborn as sns
sns.set_style('darkgrid')

# Read data

In [ ]:
c14_atm_data = pd.read_excel('./levin data set  1959-2018 raw.xlsx',
                             usecols=[2, 3],
                             header=None,
                             names=['year', 'd14C']
                             ).sort_values('year')

In [ ]:
c14_atm_data['year'].is_monotonic

In [ ]:
fig = plt.figure()
plt.plot(c14_atm_data['year'], c14_atm_data['d14C'])
plt.xlabel('year')
plt.ylabel('14C')
plt.show()

# Smooth data

In [ ]:
c14_atm_data['year'] = np.round(c14_atm_data['year']*2)/2

In [ ]:
c14_atm_data = c14_atm_data.groupby('year').mean().reset_index()

In [ ]:
fig = plt.figure()
plt.plot(c14_atm_data['year'], c14_atm_data['d14C'])
plt.xlabel('year')
plt.ylabel('14C')
plt.show()

# Extrapolate

In [ ]:
for t in [1970, 2010]:
    late_data = c14_atm_data.query('year > @t')
    fig = plt.figure()
    plt.plot(late_data['year'], late_data['d14C'])
    plt.xlabel('year')
    plt.ylabel('14C')
    plt.semilogy()
    plt.show()

In [ ]:
late_data = c14_atm_data.query('year > 2010')
fig = plt.figure()
plt.plot(late_data['year'], late_data['d14C'])
plt.xlabel('year')
plt.ylabel('14C')
#plt.semilogy()
plt.show()

In [ ]:
def f1(x, m, n):
    return m*x + n

def fit(f, p0=None):
    p = sp.optimize.curve_fit(f, late_data['year'], late_data['d14C'], p0)[0]
    return lambda x: f(x, *p)

In [ ]:
late_data = c14_atm_data.query('year > 2010')
fig = plt.figure()
plt.plot(late_data['year'], late_data['d14C'])
t = np.linspace(2010, 2020)
plt.plot(t, fit(f1)(t), label='linear')
# plt.plot(t, fit(f2, [1, 1, 2010])(t), label='exponential')
plt.xlabel('year')
plt.ylabel('14C')
plt.legend()
# plt.semilogy()
plt.show()

In [ ]:
c14_atm_data.tail()

# Extrapolate and export

In [ ]:
# c14_atm_data = pd.read_excel('../../data/14C_levin_data_until_2016_for_plotting.xlsx', names=['year', 'd14C'])

tt = np.arange(c14_atm_data['year'].max() + 0.5, 2020.1, 0.5)
cc = fit(f1)(tt)

c14_atm_data_export = c14_atm_data.copy()
j = c14_atm_data_export.index.max()
for i in np.arange(0, len(tt)-0.5, 1).astype(int):
    c14_atm_data_export.loc[j+i+1, 'year'] = tt[i]
    c14_atm_data_export.loc[j+i+1, 'd14C'] = cc[i]
    
c14_atm_data_export['d14C'] /= 1000
c14_atm_data_export = c14_atm_data_export.rename({'d14C': 'delta_14c'}, axis='columns')

In [ ]:
fig = plt.figure()
plt.plot(c14_atm_data_export['year'], c14_atm_data_export['delta_14c'], label='extrapolated')
plt.plot(c14_atm_data['year'], c14_atm_data['d14C']/1000, label='measured')

plt.xlabel('Calendar year')
plt.ylabel('$\Delta^{14}$C')

plt.legend()

# plt.savefig('plots/atm_14C_extrapolate.svg', bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure()
plt.plot(c14_atm_data_export['year'], c14_atm_data_export['delta_14c'], label='extrapolated')
plt.plot(c14_atm_data['year'], c14_atm_data['d14C']/1000, label='measured')

plt.xlabel('Calendar year')
plt.ylabel('$\Delta^{14}$C')

plt.legend()
plt.xlim(2010, None)
plt.ylim(-0.1, 0.1)
# plt.savefig('plots/atm_14C_extrapolate.svg', bbox_inches='tight')

plt.show()

In [ ]:
c14_atm_data_export.columns = ['#year', 'delta_14c']

In [ ]:
c14_atm_data_export.to_csv('./c14atm.dat', index=False, sep=' ')